## Imports & Setup

In [1]:
import sys
import yaml
import shutil
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from datetime import datetime

# Add the scripts directory to system path so we can import our modules
# Assumes notebook is in /notebooks and scripts are in /scripts
current_dir = Path.cwd()
sys.path.append(str(current_dir.parent / "scripts"))

# Import our custom robust modules
from project_paths import ROOT_DIR, CONFIG_DIR, DATA_DIR, resolve_path
import model_builder

print(f"✅ Project Root: {ROOT_DIR}")
print(f"✅ TensorFlow Version: {tf.__version__}")
print(f"✅ Devices: {tf.config.list_physical_devices()}")

2026-01-07 20:59:44.051773: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/nico/workspace/github.com/Nico-Sander/KI-Project-WS2526/.venv/lib/python3.12/site-packages/keras/src/export/tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


✅ Project Root: /home/nico/workspace/github.com/Nico-Sander/KI-Project-WS2526
✅ TensorFlow Version: 2.20.0
✅ Devices: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


## Experiment Manager

In [2]:
def setup_experiment(config):
    """
    Creates a unique run directory and backs up the config.
    Returns: run_dir (Path), log_dir (Path), checkpoint_path (Path)
    """
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    run_name = config['project']['run_name']
    
    # Create runs/YYYYMMDD_HHMMSS_run_name
    run_dir = ROOT_DIR / "runs" / f"{run_name}_{timestamp}"
    run_dir.mkdir(parents=True, exist_ok=True)
    
    # Create subdirectories
    log_dir = run_dir / "logs"
    log_dir.mkdir(exist_ok=True)
    
    checkpoint_dir = run_dir / "checkpoints"
    checkpoint_dir.mkdir(exist_ok=True)
    
    # Backup the config file
    with open(run_dir / "config.yaml", 'w') as f:
        yaml.dump(config, f)
        
    print(f"🚀 Experiment initialized at: {run_dir}")
    return run_dir, log_dir, checkpoint_dir

# Load the config
config_path = CONFIG_DIR / "model_config.yaml"
with open(config_path, 'r') as f:
    config = yaml.safe_load(f)

# Setup the run
run_dir, log_dir, checkpoint_dir = setup_experiment(config)

🚀 Experiment initialized at: /home/nico/workspace/github.com/Nico-Sander/KI-Project-WS2526/runs/run_04_deep_cnn_augmented_20260107_205951


## Data Loading Pipeline

In [3]:
# Resolve the dataset path using our helper (handles absolute/relative)
dataset_path = resolve_path(config['data']['dataset_path'])

img_size = (config['data']['img_height'], config['data']['img_width'])
batch_size = config['data']['batch_size']
color_mode = 'grayscale' if config['data']['channels'] == 1 else 'rgb'

print(f"📂 Loading data from: {dataset_path}")

# 1. Load Training Data
train_ds = tf.keras.utils.image_dataset_from_directory(
    dataset_path,
    validation_split=config['data']['validation_split'],
    subset="training",
    seed=config['project']['seed'],
    image_size=img_size,
    batch_size=batch_size,
    color_mode=color_mode
)

# 2. Load Validation Data
val_ds = tf.keras.utils.image_dataset_from_directory(
    dataset_path,
    validation_split=config['data']['validation_split'],
    subset="validation",
    seed=config['project']['seed'],
    image_size=img_size,
    batch_size=batch_size,
    color_mode=color_mode
)

# 3. Performance Optimization (The "Secret Sauce" for speed)
# cache() keeps images in memory after the first epoch
# prefetch() prepares the next batch while the GPU/CPU is working
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)

print("✅ Data Pipeline Ready")

📂 Loading data from: /home/nico/workspace/github.com/Nico-Sander/KI-Project-WS2526/data/processed_patches
Found 40000 files belonging to 2 classes.
Using 32000 files for training.


I0000 00:00:1767815998.482587   70222 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5167 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2070 SUPER, pci bus id: 0000:29:00.0, compute capability: 7.5


Found 40000 files belonging to 2 classes.
Using 8000 files for validation.
✅ Data Pipeline Ready


## Build & Compile the Model

In [4]:
# Build the model dynamically from the config
model = model_builder.build_model_from_config(config)

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ rescaling (Rescaling)           │ (None, 256, 256, 1)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ random_flip (RandomFlip)        │ (None, 256, 256, 1)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ random_rotation                 │ (None, 256, 256, 1)    │             0 │
│ (RandomRotation)                │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ random_zoom (RandomZoom)        │ (None, 256, 256, 1)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ random_translation              │ (None, 256, 256, 1)    │             0 │
│ (RandomTranslation)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gaussian_noise (GaussianNoise)  │ (None, 256, 256, 1)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ random_contrast                 │ (None, 256, 256, 1)    │             0 │
│ (RandomContrast)                │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 256, 256, 32)   │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 256, 256, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 128, 128, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 128, 128, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 128, 128, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 64, 64, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 64, 64, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 64, 64, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 32, 32, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 32, 32, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 32, 32, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 16, 16, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 16, 16, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 16, 16, 128)    │           512 │
│ (BatchNormalization)            │                        │             

 Total params: 914,177 (3.49 MB)

 Trainable params: 913,217 (3.48 MB)

 Non-trainable params: 960 (3.75 KB)

## Callbacks and Training

In [5]:
# Define Callbacks
callbacks = [
    # Stop if validation loss doesn't improve for 'patience' epochs
    tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=config['train']['callbacks']['early_stopping_patience'],
        restore_best_weights=True,
        verbose=1
    ),
    # Save the best model to the run folder
    tf.keras.callbacks.ModelCheckpoint(
        filepath=str(checkpoint_dir / "best_model.keras"),
        monitor='val_loss',
        save_best_only=True,
        verbose=1
    ),
    # Log metrics for visualization
    tf.keras.callbacks.TensorBoard(log_dir=log_dir),
    
    # Save CSV logs for easy plotting later
    tf.keras.callbacks.CSVLogger(str(run_dir / "training_log.csv"))
]

# --- NEW: Add Scheduler if enabled in config ---
if config['train']['callbacks'].get('use_lr_scheduler', False):
    scheduler = tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=config['train']['callbacks']['reduce_lr_factor'],
        patience=config['train']['callbacks']['reduce_lr_patience'],
        min_lr=config['train']['callbacks']['reduce_lr_min'],
        verbose=1  # It will print "Reducing learning rate to..."
    )
    callbacks.append(scheduler)
    print("✅ Learning Rate Scheduler: ENABLED")

# START TRAINING
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=config['train']['epochs'],
    callbacks=callbacks
)

# Save the final model just in case
model.save(run_dir / "final_model.keras")
print("🏁 Training Complete!")

✅ Learning Rate Scheduler: ENABLED
Epoch 1/50


2026-01-07 21:00:34.799710: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:473] Loaded cuDNN version 91701
2026-01-07 21:00:37.022739: W external/local_xla/xla/tsl/framework/bfc_allocator.cc:310] Allocator (GPU_0_bfc) ran out of memory trying to allocate 3.29GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.


500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 187ms/step - accuracy: 0.6900 - loss: 0.6070
Epoch 1: val_loss improved from None to 0.56970, saving model to /home/nico/workspace/github.com/Nico-Sander/KI-Project-WS2526/runs/run_04_deep_cnn_augmented_20260107_205951/checkpoints/best_model.keras

Epoch 1: finished saving model to /home/nico/workspace/github.com/Nico-Sander/KI-Project-WS2526/runs/run_04_deep_cnn_augmented_20260107_205951/checkpoints/best_model.keras
500/500 ━━━━━━━━━━━━━━━━━━━━ 114s 197ms/step - accuracy: 0.7469 - loss: 0.5069 - val_accuracy: 0.6591 - val_loss: 0.5697 - learning_rate: 5.0000e-04
Epoch 2/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 181ms/step - accuracy: 0.8374 - loss: 0.3740
Epoch 2: val_loss improved from 0.56970 to 0.46519, saving model to /home/nico/workspace/github.com/Nico-Sander/KI-Project-WS2526/runs/run_04_deep_cnn_augmented_20260107_205951/checkpoints/best_model.keras

Epoch 2: finished saving model to /home/nico/workspace/github.com/Nico-Sander/KI-Project-WS2526/runs/ru

## Visualization

In [1]:
# Plotting the results
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(len(acc))

plt.figure(figsize=(12, 4))

# Accuracy Plot
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

# Loss Plot
plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')

plt.show()

NameError: name 'history' is not defined